In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import sys 
sys.path.append("..")

In [ ]:
import numpy as np
import pandas as pd
from track_reid.reid_processor import ReidProcessor

In [ ]:
def bounding_box_distance(obj1, obj2):
    # Get the bounding boxes from the Metadata of each TrackedObject
    bbox1 = obj1.metadata.bbox
    bbox2 = obj2.metadata.bbox

    # Calculate the Euclidean distance between the centers of the bounding boxes
    center1 = ((bbox1[0] + bbox1[2]) / 2, (bbox1[1] + bbox1[3]) / 2)
    center2 = ((bbox2[0] + bbox2[2]) / 2, (bbox2[1] + bbox2[3]) / 2)
    distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)

    return distance

def select_by_category(obj1, obj2):
    # Compare the categories of the two objects
    return 1 if obj1.category == obj2.category else 0

In [ ]:
# Example usage:
data = np.array([
    [1, 1, "car", 100, 200, 300, 400, 0.9],
    [1, 2, "person", 50, 150, 200, 400, 0.8],
    [2, 1, "truck", 120, 220, 320, 420, 0.95],
    [2, 2, "person", 60, 160, 220, 420, 0.85],
    [3, 1, "car", 110, 210, 310, 410, 0.91],
    [3, 3, "person", 61, 170, 220, 420, 0.91],
    [3, 4, "car", 60, 160, 220, 420, 0.91],
    [3, 6, "person", 60, 160, 220, 420, 0.91],
    [4, 1, "truck", 130, 230, 330, 430, 0.92],
    [4, 2, "person", 65, 165, 225, 425, 0.83],
    [5, 1, "car", 115, 215, 315, 415, 0.93],
    [5, 2, "person", 57, 157, 207, 407, 0.84],
    [5, 4, "car", 60, 160, 220, 420, 0.91],
    [5, 8, "person", 60, 160, 220, 420, 0.91],
])


In [ ]:
processor = ReidProcessor(filter_confidence_threshold=0.4, 
                          filter_time_threshold=0,
                          cost_function=bounding_box_distance,
                          selection_function=select_by_category,
                          max_attempt_to_rematch=0,
                          max_frames_to_rematch=100)


columns = ['frame_id', 'object_id', 'category', 'x1', 'y1', 'x2', 'y2', 'confidence']
df = pd.DataFrame(data, columns=columns)
# Convert numerical columns to the appropriate data type
df[['frame_id', 'object_id', 'x1', 'y1', 'x2', 'y2']] = df[['frame_id', 'object_id', 'x1', 'y1', 'x2', 'y2']].astype(int)
df['confidence'] = df['confidence'].astype(float)


for frame_id, frame_data in df.groupby("frame_id"):

    bytetrack_output = frame_data.values
    if bytetrack_output.ndim == 1 : 
        bytetrack_output = np.expand_dims(bytetrack_output, 0)

    results = processor.update(bytetrack_output)


In [ ]:
print(processor.all_tracked_objects[0])

In [ ]:
print(processor.all_tracked_objects[0].metadata)